<a href="https://colab.research.google.com/github/FedericoBorbiconi/labo2025ros/blob/main/FB_EnsemblesArbolesDecision_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Ensembles de Arboles de Decision

## 5.4 GBDT LightGBM

La técnica de Gradient Boosting fue creada por Jerome H. Friedman en 1999 - 2001
<br>Se implementaron librerías ineficientes
<br>En 2016 se crea XGBoost, en 2017 LightGBM

El Gradient Boosting of Decision Trees es un ensemble de árboles de decisión, para un nuevo registro la predicción se hace sumando el score que cada arbol asigna a ese registro.

En GBDT la construccion de los árboles es secuencial, ya que el arbol n-simo se genera para predecir el error del modelo conformado por los  n-1 arboles previos, aunque sea un arbol de clasificación lo que se predice es un numero real mediante un arbol de regresión.


<br>Qué tipo de perturbaciones se realiza LightGBM

*   Se perturba el dataset, seleccionando para cada arbol un subconjunto de las columnas.
*   El algortimo de arbol de decisión no presenta perturbaciones

Cada arbolito de LightGBM se entrena sobre un dataset perturbado, que en principio posee :
* todos los registros del dataset original
* solo un porcentaje *feature_fraction* de las columnas originales del dataset

#### 5.4.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 5.4.2  LightGBM, una corrida

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2305164,123.2,3948767,210.9,3948767,210.9
Vcells,55344600,422.3,139047760,1060.9,123613410,943.1


In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

Aqui debe cargar SU semilla primigenia

In [ ]:
PARAM <- list()
PARAM$experimento <- 5427
PARAM$semilla_primigenia <- 777743

# estos hiperparametros de LightGBM surgieron de una Bayesian Optimization
PARAM$lgb$num_iterations <- 2000  # cantidad de arbolitos
PARAM$lgb$learning_rate <- 0.24005883
PARAM$lgb$feature_fraction <- 0.8143109
PARAM$lgb$min_data_in_leaf <- 1354
PARAM$lgb$num_leaves <- 22
PARAM$lgb$max_bin <- 31

# Agrego los míos
PARAM$lgb$max_depth <- 9
PARAM$lgb$min_gain_to_split <- 7.216148
PARAM$lgb$bagging_fraction <- 0.8873539
PARAM$lgb$bagging_freq <- 2
PARAM$lgb$lambda_l1 <- 35.77308887
PARAM$lgb$lambda_l2 <- 43.7868426


In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors = TRUE)

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
# set trabaja con la clase  POS = { BAJA+1, BAJA+2 }
# esta estrategia es MUY importante
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2", "BAJA+1"), 1L, 0L)]

In [ ]:
# los campos que se van a utilizar
campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [ ]:
# establezco donde entreno
dataset[, train := 0L]
dataset[foto_mes %in% c(202107), train := 1L]

In [ ]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[train == 1L, campos_buenos, with = FALSE]),
  label= dataset[train == 1L, clase01]
)

In [ ]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria

modelo <- lgb.train(
  data= dtrain,
  param= list(
    objective= "binary",
    max_bin= PARAM$lgb$max_bin,
    learning_rate= PARAM$lgb$learning_rate,
    num_iterations= PARAM$lgb$num_iterations,
    num_leaves= PARAM$lgb$num_leaves,
    min_data_in_leaf= PARAM$lgb$min_data_in_leaf,
    feature_fraction= PARAM$lgb$feature_fraction,
    seed= PARAM$semilla_primigenia,
    max_depth = PARAM$lgb$max_depth,
    min_gain_to_split = PARAM$lgb$min_gain_to_split,
    bagging_fraction = PARAM$lgb$bagging_fraction,
    bagging_freq = PARAM$lgb$bagging_freq,
    lambda_l1 = PARAM$lgb$lambda_l1,
    lambda_l2 = PARAM$lgb$lambda_l2
  )
)


[LightGBM] [Info] Number of positive: 2385, number of negative: 162211
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.117525 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3316
[LightGBM] [Info] Number of data points in the train set: 164596, number of used features: 137
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014490 -> initscore=-4.219699
[LightGBM] [Info] Start training from score -4.219699
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [ ]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)


In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo, "modelo.txt" )

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)


In [ ]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file = "prediccion.txt",
  sep = "\t"
)

In [ ]:
# subidas a Kaggle
# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

cortes <- seq(9000, 13500, by = 500)

for (envios in cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0("KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # subida a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- "-c labo-i-2025-rosario"
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'num_iterations=", PARAM$lgb$num_iterations,
    "  learning_rate=", PARAM$lgb$learning_rate,
    "  feature_fraction=", PARAM$lgb$feature_fraction,
    "  min_data_in_leaf=", PARAM$lgb$min_data_in_leaf,
    "  num_leaves=",PARAM$lgb$num_leaves,
    "  max_bin=", PARAM$lgb$max_bin,
  "'" )

  linea <- paste( comando, competencia, arch, mensaje)
  salida <- system(linea, intern=TRUE)
  cat(salida)
}

Successfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 Rosario



---



### 5.4.3  LightGBM  optimizacion de hiperparámetros

La optimizacion de los hiperparámetros de LightGBM mediante el método de optimizacion bayesiana será su *caballito de batalla* durante la asignatura !

limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649922,34.8,1439337,76.9,1070521,57.2
Vcells,1209991,9.3,8388608,64.0,1924971,14.7


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Loading required package: rlist

Loading required package: lightgbm

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




Aqui debe cargar SU semilla primigenia - https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/

In [3]:
PARAM <- list()
PARAM$experimento <- 5430
PARAM$semilla_primigenia <- 277073

# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos
PARAM$trainingstrategy$undersampling <- 1.0

PARAM$hyperparametertuning$iteraciones <- 200
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 117000
PARAM$hyperparametertuning$NEG_ganancia <- -3000

# Aqui se cargan los bordes de los hiperparametros
PARAM$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.4),
  makeIntegerParam("num_leaves", lower = 4L, upper = 128L),
  makeIntegerParam("max_depth", lower = 3L, upper = 16L),
  makeIntegerParam("min_data_in_leaf", lower = 5L, upper = 4000L),
  makeNumericParam("min_gain_to_split", lower = 0, upper = 15),
  makeNumericParam("bagging_fraction", lower = 0.5, upper = 1),
  makeIntegerParam("bagging_freq", lower = 1L, upper = 5L),
  makeNumericParam("feature_fraction", lower = 0.1, upper = 1.0),
  makeNumericParam("lambda_l1", lower = 0.001, upper = 100),
  makeNumericParam("lambda_l2", lower = 0.001, upper = 100),
  makeIntegerParam("envios", lower = 5000L, upper = 15000L)
)

In [4]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch = NA, folder = "./work/",
    ext = ".txt", verbose = TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse = "\t"), "\n"
      )

      cat(linea, file = archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file = archivo, append = TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [5]:
# esta funcion calcula internamente la ganancia de la prediccion probs
# es llamada por lightgbm luego de construir cada  arbolito

fganancia_logistic_lightgbm <- function(probs, datos) {
  vpesos <- get_field(datos, "weight")

  # vector de ganancias
  vgan <- ifelse(vpesos == 1.0000002, PARAM$hyperparametertuning$POS_ganancia,
    ifelse(vpesos == 1.0000001, PARAM$hyperparametertuning$NEG_ganancia,
      PARAM$hyperparametertuning$NEG_ganancia /
        PARAM$trainingstrategy$undersampling
    )
  )

  tbl <- as.data.table(list("vprobs" = probs, "vgan" = vgan))
  setorder(tbl, -vprobs)
  ganancia <- tbl[1:GLOBAL_envios, sum(vgan)]

  return(list(
    "name" = "ganancia",
    "value" = ganancia,
    "higher_better" = TRUE
  ))
}


In [6]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales,
# la semilla del mal ...


EstimarGanancia_lightgbm <- function(x) {
  gc() # libero memoria

  # llevo el registro de la iteracion por la que voy
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # para usar en fganancia_logistic_lightgbm
  # asigno la variable global
  GLOBAL_envios <<- as.integer(x$envios / PARAM$hyperparametertuning$xval_folds)

  # cantidad de folds para cross validation
  kfolds <- PARAM$hyperparametertuning$xval_folds

  param_basicos <- list(
    objective= "binary",
    metric= "custom",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    max_bin= 31, # por ahora, lo dejo fijo
    num_iterations= 9999, # valor grande, lo limita early_stopping_rounds
    force_row_wise= TRUE, # para evitar warning
    seed= ksemilla_azar1
  )

  # el parametro discolo, que depende de otro
  param_variable <- list(
    early_stopping_rounds =
      as.integer(50 + 5 / x$learning_rate)
  )

  param_completo <- c(param_basicos, param_variable, x)

  set.seed(ksemilla_azar1)
  modelocv <- lgb.cv(
    data= dtrain,
    eval= fganancia_logistic_lightgbm,
    stratified= TRUE, # sobre el cross validation
    nfold= kfolds, # folds del cross validation
    param= param_completo,
    verbose= -100
  )

  # obtengo la ganancia
  ganancia <- unlist(modelocv$record_evals$valid$ganancia$eval)[modelocv$best_iter]

  ganancia_normalizada <- ganancia * kfolds # normailizo la ganancia

  # asigno el mejor num_iterations
  param_completo$num_iterations <- modelocv$best_iter
  # elimino de la lista el componente
  param_completo["early_stopping_rounds"] <- NULL


  # el lenguaje R permite asignarle ATRIBUTOS a cualquier variable
  # esta es la forma de devolver un parametro extra
  attr(ganancia_normalizada, "extras") <-
    list("num_iterations" = modelocv$best_iter)

  # logueo
  xx <- param_completo
  xx$ganancia <- ganancia_normalizada # le agrego la ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = klog)

  # Voy registrando la importancia de variables
  if (ganancia_normalizada > GLOBAL_gananciamax) {
    GLOBAL_gananciamax <<- ganancia_normalizada
    modelo <- lgb.train(
      data = dtrain,
      param = param_completo,
      verbose = -100
    )

    tb_importancia <- as.data.table(lgb.importance(modelo))
    archivo_importancia <- paste0("impo_", GLOBAL_iteracion, ".txt")
    fwrite(tb_importancia,
      file = archivo_importancia,
      sep = "\t" )

    loguear(xx, arch = klog_mejor)
  }

  return(ganancia_normalizada)
}


aqui se inicia el programa

In [7]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [8]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )
ksemilla_azar1 <- PARAM$semillas[1]
ksemilla_azar2 <- PARAM$semillas[2]


In [9]:
# en estos archivos quedan los resultados

kbayesiana <- paste0(PARAM$experimento, ".RDATA")
klog <- paste0(PARAM$experimento, ".txt")
klog_mejor <- paste0(PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_gananciamax <- -1 # inicializo la variable global

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_gananciamax <- tabla_log[, max(ganancia)]
}


In [10]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [11]:
dataset <- dataset[foto_mes %in% c(202107)]

In [12]:
# paso la clase a binaria que tome valores {0,1}  enteros
dataset[
  foto_mes %in% c(202107),
  clase01 := ifelse(clase_ternaria == "CONTINUA", 0L, 1L)
]

In [13]:
# los campos que se van a utilizar
campos_buenos <- setdiff(
  colnames(dataset),
  c("clase_ternaria", "clase01", "azar", "training")
)


In [14]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(ksemilla_azar2)
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]
dataset[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [15]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label = dataset[training == 1L, clase01],
  weight = dataset[training == 1L, ifelse(clase_ternaria == "BAJA+2", 1.0000002, ifelse(clase_ternaria == "BAJA+1", 1.0000001, 1.0))],
  free_raw_data = FALSE
)


In [16]:
# Aqui comienza la configuracion de la Bayesian Optimization
funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar, # la funcion que voy a maximizar
  minimize = FALSE, # estoy Maximizando la ganancia
  noisy = TRUE,
  par.set = PARAM$hs, # definido al comienzo del programa
  has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600, # se graba cada 600 segundos
  save.file.path = kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters = PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type = "se",
  covtype = "matern3_2",
  control = list(trace = TRUE)
)


In [17]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
  run <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.



20250606 010403	binary	custom	TRUE	TRUE	FALSE	-100	31	32	TRUE	441187	0.193382371041172	89	10	882	3.56845915635032	0.611452846864598	1	0.34164546784331	61.4488810643636	52.2922131502295	9756	52935000	1
20250606 010406	binary	custom	TRUE	TRUE	FALSE	-100	31	32	TRUE	441187	0.193382371041172	89	10	882	3.56845915635032	0.611452846864598	1	0.34164546784331	61.4488810643636	52.2922131502295	9756	52935000	1
20250606 010422	binary	custom	TRUE	TRUE	FALSE	-100	31	18	TRUE	441187	0.221038907354761	25	7	1790	7.51332865628435	0.776557794406968	4	0.49117677801458	52.2442107247923	31.7611578839845	11848	52665000	2
20250606 010442	binary	custom	TRUE	TRUE	FALSE	-100	31	32	TRUE	441187	0.119844739559763	70	8	3183	5.15506148327735	0.601169478886401	2	0.556543244200822	56.2873000319151	35.9475617846607	8617	50715000	3
20250606 010506	binary	custom	TRUE	TRUE	FALSE	-100	31	25	TRUE	441187	0.0894011040133509	33	16	2290	10.6281252812848	0.81573232633739	3	0.484436591511952	59.4931122868016	80.5372600754472	11031	5

[mbo] 0: learning_rate=0.193; num_leaves=89; max_depth=10; min_data_in_leaf=882; min_gain_to_split=3.57; bagging_fraction=0.611; bagging_freq=1; feature_fraction=0.342; lambda_l1=61.4; lambda_l2=52.3; envios=9756 : y = 5.29e+07 : 25.4 secs : initdesign

[mbo] 0: learning_rate=0.221; num_leaves=25; max_depth=7; min_data_in_leaf=1790; min_gain_to_split=7.51; bagging_fraction=0.777; bagging_freq=4; feature_fraction=0.491; lambda_l1=52.2; lambda_l2=31.8; envios=11848 : y = 5.27e+07 : 16.6 secs : initdesign

[mbo] 0: learning_rate=0.12; num_leaves=70; max_depth=8; min_data_in_leaf=3183; min_gain_to_split=5.16; bagging_fraction=0.601; bagging_freq=2; feature_fraction=0.557; lambda_l1=56.3; lambda_l2=35.9; envios=8617 : y = 5.07e+07 : 18.9 secs : initdesign

[mbo] 0: learning_rate=0.0894; num_leaves=33; max_depth=16; min_data_in_leaf=2290; min_gain_to_split=10.6; bagging_fraction=0.816; bagging_freq=3; feature_fraction=0.484; lambda_l1=59.5; lambda_l2=80.5; envios=11031 : y = 5.26e+07 : 23.8 

20250606 012057	binary	custom	TRUE	TRUE	FALSE	-100	31	386	TRUE	441187	0.059490851659983	50	9	1830	0.517944752267783	0.839228726336647	3	0.934580172894294	2.66748139710232	50.7293697582692	11905	58365000	45
20250606 012112	binary	custom	TRUE	TRUE	FALSE	-100	31	386	TRUE	441187	0.059490851659983	50	9	1830	0.517944752267783	0.839228726336647	3	0.934580172894294	2.66748139710232	50.7293697582692	11905	58365000	45


[mbo] 1: learning_rate=0.0595; num_leaves=50; max_depth=9; min_data_in_leaf=1830; min_gain_to_split=0.518; bagging_fraction=0.839; bagging_freq=3; feature_fraction=0.935; lambda_l1=2.67; lambda_l2=50.7; envios=11905 : y = 5.84e+07 : 103.1 secs : infill_ei



...

[mbo] 198: learning_rate=0.0249; num_leaves=59; max_depth=8; min_data_in_leaf=3584; min_gain_to_split=0.0222; bagging_fraction=0.914; bagging_freq=4; feature_fraction=0.366; lambda_l1=0.0134; lambda_l2=44.7; envios=10002 : y = 6.01e+07 : 297.1 secs : infill_ei

Saved the current state after iteration 199 in the file 5430.RDATA.



20250606 055459	binary	custom	TRUE	TRUE	FALSE	-100	31	193	TRUE	441187	0.336366404527548	110	10	970	0.00467854408557408	0.988775419614099	3	0.408341784861758	9.33904599406665	67.5420952202244	11542	61140000	243


[mbo] 199: learning_rate=0.336; num_leaves=110; max_depth=10; min_data_in_leaf=970; min_gain_to_split=0.00468; bagging_fraction=0.989; bagging_freq=3; feature_fraction=0.408; lambda_l1=9.34; lambda_l2=67.5; envios=11542 : y = 6.11e+07 : 66.8 secs : infill_ei



20250606 055920	binary	custom	TRUE	TRUE	FALSE	-100	31	613	TRUE	441187	0.0123740130423783	64	8	2872	0.0143471002416779	0.932900259849122	4	0.500310752738833	0.0201739698109305	66.1955224024628	11696	58035000	244


[mbo] 200: learning_rate=0.0124; num_leaves=64; max_depth=8; min_data_in_leaf=2872; min_gain_to_split=0.0143; bagging_fraction=0.933; bagging_freq=4; feature_fraction=0.5; lambda_l1=0.0202; lambda_l2=66.2; envios=11696 : y = 5.8e+07 : 254.4 secs : infill_ei

Saved the final state in the file 5430.RDATA





---

